In [19]:
import numpy as np
from scipy.optimize import least_squares


In [20]:
import numpy as np
import pandas as pd

# Load the dataset
df = pd.read_csv('datasets/acc_data_inseconds.csv')

# Ensure there are no more than 3000 rows
df = df.iloc[:3000]

# Extract only the 'X', 'Y', 'Z' columns
data = df[['X', 'Y', 'Z']]

chunk_size = 100
num_chunks = int(len(df) / chunk_size)

average_variances = np.zeros(3)  # There are 3 dimensions: X, Y, Z

# Calculate the average variance for each dimension
for i in range(3):
    column_variances = [
        np.var(data.iloc[j * chunk_size:(j + 1) * chunk_size, i]) for j in range(num_chunks)
    ]
    average_variances[i] = np.mean(column_variances)

# Sort the indices of average_variances in descending order of variance
ordered_indices = np.argsort(average_variances)[::-1]

# Create a dictionary mapping old column names to new ones based on variance
columns_map = {data.columns[i]: ('h', 'a', 's')[index] for i, index in enumerate(ordered_indices)}

# Rename the columns using the mapping dictionary
data_renamed = data.rename(columns=columns_map)

print("Dataframe with renamed columns based on variance:")
print(data_renamed.head())


Dataframe with renamed columns based on variance:
          s         a         h
0  2.343741  2.299612  9.312727
1  2.291577  2.325804  8.796689
2  2.088251  2.305492  8.465601
3  1.862871  2.251594  8.339330
4  1.898391  2.154092  8.541755


In [21]:
# Given system parameters (for simulation)
g = 9.81  # gravity constant


In [22]:
n_samples = len(df)
theta = np.random.rand(n_samples) * 2 * np.pi  # Uniformly distributed angles
h = data_renamed['h'].values  # Random vertical accelerations
a = data_renamed['a'].values  # Random horizontal accelerations


In [23]:
# Calculate actual horizontal and vertical accelerations based on given dynamics
A_H = (h - g) * np.sin(theta) + a * np.cos(theta)
A_v = (h - g) * np.cos(theta) - a * np.sin(theta)


In [24]:
# Build the library of functions (nonlinear transformation)
# In this case, we have sin(theta), cos(theta), h*sin(theta), h*cos(theta), a*sin(theta), and a*cos(theta)
library = np.column_stack((np.sin(theta), np.cos(theta), h * np.sin(theta), h * np.cos(theta), a * np.sin(theta), a * np.cos(theta)))


In [31]:
# Sparse regression function using least squares and manual sparsity
def sparse_regression(library, target, sparsity_threshold=0.5):
    # Least squares fit
    res = least_squares(lambda coef: library @ coef - target, np.zeros(library.shape[1]))
    coef = res.x
    # Enforce sparsity by zeroing out small coefficients
    sparse_coef = coef * (np.abs(coef) > sparsity_threshold)
    return sparse_coef




In [32]:
# Perform sparse regression for horizontal and vertical accelerations
coefficients_H = sparse_regression(library, A_H)
coefficients_v = sparse_regression(library, A_v)


In [33]:
print("Identified coefficients for A_H(t):", coefficients_H)
print("Identified coefficients for A_v(t):", coefficients_v)


Identified coefficients for A_H(t): [-9.81  0.    1.   -0.    0.    1.  ]
Identified coefficients for A_v(t): [ 0.   -9.81  0.    1.   -1.    0.  ]
